In [26]:
from public_data.models import Commune, Cerema, Epci, Scot
from django.contrib.gis.db.models import Union

In [6]:
initials = {c: (e, s) for c, e, s in Commune.objects.all().values_list("insee", "epci__source_id", "scot__name")}
updated = {c: (e, s) for c, e, s in Cerema.objects.all().values_list("city_insee", "epci_id", "scot")}
len(initials), len(updated)

(34826, 34806)

In [9]:
city_to_delete = [c for c in initials.keys() if c not in updated]
len(city_to_delete)

21

In [10]:
city_to_create = [c for c in updated.keys() if c not in initials]
city_to_create

['27058']

In [21]:
city_with_epci_to_update = [c for c, e in initials.items() if c in updated and e[0] != updated[c][0]]
len(city_with_epci_to_update)

194

In [23]:
city_with_scot_to_update = [c for c, e in initials.items() if c in updated and e[1] != updated[c][1]]
len(city_with_scot_to_update)

29540

In [25]:
Cerema.objects.all().values_list("scot", flat=True).distinct()

<QuerySet ['SCoT du Pays de Brie et Champagne', 'SCoT du Gard Rhodanien', 'SCoT Gevaudan Lozère', "SCoT de l'aire métropolitaine bordelaise", 'SCoT Cap Atlantique', 'SCoT Littoral Sud', 'SCoT Golfe du Morbihan - Vannes agglomération', 'SCoT du Pays de Valençay en Berry', 'SCoT Nord Isere', "SCoT de l'Agglomération d'Angoulême", "SCoT du Pays de l'Agenais", 'SCoT du Bassin de Vie de Coulommiers', 'SCoT du Soissonnais en Valois', "SCoT du Pays de l'auxois Morvan", 'SCoT du Haut Bearn', 'SCoT du Pays de Langres', 'SCoT Roannais', 'SCoT du Grand Nevers', 'SCoT des Pyrénées Catalanes', 'SCoT du Pays de Morlaix', '...(remaining elements truncated)...']>

In [27]:
new_epci = Cerema.objects.all().values("epci_id", "epci_name").annotate(geom=Union("mpoly"))

<QuerySet [{'epci_id': '200000172', 'epci_name': 'CC Faucigny-Glières', 'geom': <Polygon object at 0x7fdcf5d0d7b0>}, {'epci_id': '200000438', 'epci_name': 'CC du Pays de Pontchâteau Saint-Gildas-des-Bois', 'geom': <Polygon object at 0x7fdcf5d0d840>}, {'epci_id': '200000545', 'epci_name': 'CC des Portes de Romilly-sur-Seine', 'geom': <Polygon object at 0x7fdcf5d0d8d0>}, {'epci_id': '200000628', 'epci_name': 'CC Rhône Lez Provence', 'geom': <Polygon object at 0x7fdcf5d0d960>}, {'epci_id': '200000800', 'epci_name': 'CC Cœur de Sologne', 'geom': <Polygon object at 0x7fdcf5d0d9f0>}, {'epci_id': '200000925', 'epci_name': 'CC de la Plaine Dijonnaise', 'geom': <Polygon object at 0x7fdcf5d0da80>}, {'epci_id': '200000933', 'epci_name': 'CC Sauldre et Sologne', 'geom': <Polygon object at 0x7fdcf5d0db10>}, {'epci_id': '200004802', 'epci_name': 'CC du Pays de Fayence', 'geom': <Polygon object at 0x7fdcf5d0dba0>}, {'epci_id': '200005932', 'epci_name': 'CC des Portes de Sologne', 'geom': <Polygon obj

In [35]:
a = [1,2,3]
while a:
    b = a.pop()
    print(b)
print(a)

3
2
1
[]


In [31]:
from django.db.models import Count, IntegerField

Cerema.objects.all().values("scot").annotate(
    geom=Union("mpoly"),
    num_regions=Count("region_id", distinct=True),
    num_depts=Count("dept_id", distinct=True),
)

<QuerySet [{'scot': 'Communautés de communes Usses et Rhône', 'geom': <Polygon object at 0x7fdcf7480490>, 'num_regions': 1, 'num_depts': 2}, {'scot': 'Redon Agglomeration', 'geom': <Polygon object at 0x7fdcf5d0e410>, 'num_regions': 2, 'num_depts': 3}, {'scot': 'SCoT  Corbieres Salanque Mediterranée', 'geom': <Polygon object at 0x7fdcf5d0e4a0>, 'num_regions': 1, 'num_depts': 2}, {'scot': 'SCoT AEC du Sud-Corse', 'geom': <MultiPolygon object at 0x7fdcf5d0e530>, 'num_regions': 1, 'num_depts': 1}, {'scot': 'SCoT AUTAN COCAGNE', 'geom': <Polygon object at 0x7fdcf5d0e5c0>, 'num_regions': 1, 'num_depts': 1}, {'scot': 'SCoT Adour Chalosse Tursan', 'geom': <Polygon object at 0x7fdcf5d0e650>, 'num_regions': 2, 'num_depts': 2}, {'scot': 'SCoT Aix-Marseille-Provence', 'geom': <MultiPolygon object at 0x7fdcf5d0e6e0>, 'num_regions': 1, 'num_depts': 3}, {'scot': "SCoT Alpes d'Azur", 'geom': <Polygon object at 0x7fdcf5d0e770>, 'num_regions': 1, 'num_depts': 1}, {'scot': 'SCoT Alpes-Provences-Verdon', 